In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('news.csv')

In [4]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [24]:
X = df.drop(['text','Unnamed: 0','label'],axis=1)

In [25]:
X

,title
0,You Can Smell Hillary’s Fear
1,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy
3,Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matters
...,...
6330,State Department says it can't find emails fro...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...
6333,"In Ethiopia, Obama seeks progress on peace, se..."


In [26]:
y = df['label']
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [27]:
X.shape

(6335, 1)

In [28]:
y.shape

(6335,)

In [29]:
import tensorflow as tf

In [30]:
tf.__version__

'2.4.1'

In [31]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [32]:
voc_size = 3000

In [33]:
# Data Preprocessing

In [34]:
import nltk
import re
from nltk.corpus import stopwords

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/utkarshkharb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
news_titles = X.copy()

In [37]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news_titles)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', news_titles['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074


3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740


5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398


In [38]:
corpus

['smell hillari fear',
 'watch exact moment paul ryan commit polit suicid trump ralli video',
 'kerri go pari gestur sympathi',
 'berni support twitter erupt anger dnc tri warn',
 'battl new york primari matter',
 'tehran usa',
 'girl horrifi watch boyfriend left facetim',
 'britain schindler die',
 'fact check trump clinton command chief forum',
 'iran reportedli make new push uranium concess nuclear talk',
 'three clinton iowa glimps fire elud hillari clinton campaign',
 'donald trump shockingli weak deleg game somehow got even wors',
 'strong solar storm tech risk today news oct video',
 'way america prepar world war',
 'trump take cruz lightli',
 'women lead differ',
 'shock michel obama hillari caught glamor date rape promot',
 'hillari clinton huge troubl america notic sick thing hidden pictur liberti writer news',
 'iran bill obama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope trump propos ban muslim',
 'episod sunday wire hail deplor special g

In [40]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2688, 232, 1397],
 [1575, 1791, 761, 2025, 567, 1646, 1089, 1028, 1732, 1379, 2171],
 [2820, 2591, 2897, 23, 1211],
 [447, 2051, 2548, 1533, 410, 577, 1082, 1565],
 [2593, 8, 2864, 2987, 224],
 [884, 716],
 [1043, 2155, 1575, 2255, 465, 523],
 [661, 931, 2155],
 [224, 2894, 1732, 2134, 2274, 2696, 2865],
 [1460, 1196, 2028, 8, 109, 1040, 635, 844, 1335],
 [2961, 2134, 1357, 107, 753, 303, 232, 2134, 301],
 [2251, 1732, 1533, 2556, 1122, 2841, 1252, 2693, 2262, 2333],
 [1008, 2019, 2044, 1880, 2563, 577, 857, 1516, 2171],
 [460, 2265, 1372, 2540, 2944],
 [1732, 1131, 462, 330],
 [1346, 1651, 1410],
 [1402, 1726, 1287, 232, 873, 1960, 1577, 2416, 190],
 [232, 2134, 2057, 305, 2265, 2177, 223, 2219, 2683, 793, 1922, 434, 857],
 [1460, 528, 1287, 2285],
 [1673, 315, 1563, 2385, 1927, 1255, 2265],
 [2046, 2076, 1732, 2470, 697, 1165],
 [1200, 2808, 2658, 2613, 830, 372, 1379, 1192, 2065],
 [232, 2134, 2028, 2706, 2242, 183, 12],
 [8, 62, 2357, 1406, 1154, 1769, 2752, 613],
 [359, 1522, 31

In [46]:
pd1 = pd.DataFrame(corpus)

In [60]:
pd1['text_length'] = pd1[0].apply(lambda x: len(x.split())) #finding out the average word length of news titles

In [61]:
pd1

,0,text_length
0,smell hillari fear,3
1,watch exact moment paul ryan commit polit suic...,11
2,kerri go pari gestur sympathi,5
3,berni support twitter erupt anger dnc tri warn,8
4,battl new york primari matter,5
...,...,...
6330,state depart say find email clinton specialist,7
6331,p pb stand plutocrat pentagon,5
6332,anti trump protest tool oligarchi inform,6
6333,ethiopia obama seek progress peac secur east a...,8


In [63]:
pd1.describe()

,text_length
count,6335.000000
mean,7.355170
std,2.659585
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,26.000000


In [98]:
sent_length= 14 # in accordance with the above
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2688  232 1397]
 [   0    0    0 ... 1732 1379 2171]
 [   0    0    0 ... 2897   23 1211]
 ...
 [   0    0    0 ... 1274 2802 2121]
 [   0    0    0 ... 2458  406 1062]
 [   0    0    0 ...  935 1732  224]]


In [99]:
## Creating model
embedding_vector_features= 100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 14, 100)           300000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 380,501
Trainable params: 380,501
Non-trainable params: 0
_________________________________________________________________
None


In [100]:
y = df['label']

In [101]:
y

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [102]:
temp = y.apply(lambda x: 1 if x=="REAL" else 0)

In [103]:
temp

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64

In [104]:
embedded_docs

array([[   0,    0,    0, ..., 2688,  232, 1397],
       [   0,    0,    0, ..., 1732, 1379, 2171],
       [   0,    0,    0, ..., 2897,   23, 1211],
       ...,
       [   0,    0,    0, ..., 1274, 2802, 2121],
       [   0,    0,    0, ..., 2458,  406, 1062],
       [   0,    0,    0, ...,  935, 1732,  224]], dtype=int32)

In [105]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(temp)

In [106]:
X_final.shape,y_final.shape

((6335, 14), (6335,))

In [107]:
y_final

array([0, 0, 1, ..., 0, 1, 1])

In [108]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [109]:
X_train

array([[   0,    0, 1306, ...,  232, 2385, 1118],
       [   0,    0,    0, ..., 2169, 1629, 2987],
       [   0,    0,    0, ..., 1461,  294, 2919],
       ...,
       [   0,    0,    0, ..., 1323, 2751, 2311],
       [   0,    0,    0, ..., 1603,  378, 2756],
       [   0,    0,    0, ...,  889, 1798, 2740]], dtype=int32)

In [110]:
X_test

array([[   0,    0,    0, ...,  915,  696, 2335],
       [   0,    0,    0, ..., 2987, 2743, 2453],
       [   0,    0,    0, ..., 2061,  232, 2134],
       ...,
       [   0,    0,    0, ..., 2379, 2155, 2783],
       [   0,    0,    0, ..., 2719, 2810, 2333],
       [   0,    0,    0, ..., 1605, 1566,  376]], dtype=int32)

In [111]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [112]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [113]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
67/67 [==============================] - 4s 34ms/step - loss: 0.6702 - accuracy: 0.5643 - val_loss: 0.5044 - val_accuracy: 0.7633
Epoch 2/20
67/67 [==============================] - 2s 26ms/step - loss: 0.3929 - accuracy: 0.8308 - val_loss: 0.5008 - val_accuracy: 0.7633
Epoch 3/20
67/67 [==============================] - 2s 26ms/step - loss: 0.2648 - accuracy: 0.8905 - val_loss: 0.5391 - val_accuracy: 0.7642
Epoch 4/20
67/67 [==============================] - 2s 26ms/step - loss: 0.1972 - accuracy: 0.9273 - val_loss: 0.6251 - val_accuracy: 0.7551
Epoch 5/20
67/67 [==============================] - 2s 26ms/step - loss: 0.1549 - accuracy: 0.9376 - val_loss: 0.7603 - val_accuracy: 0.7580
Epoch 6/20
67/67 [==============================] - 2s 29ms/step - loss: 0.1119 - accuracy: 0.9625 - val_loss: 0.9645 - val_accuracy: 0.7542
Epoch 7/20
67/67 [==============================] - 2s 30ms/step - loss: 0.0758 - accuracy: 0.9724 - val_loss: 1.1722 - val_accuracy: 0.7542
Epoch 8/20
67

In [114]:
y_pred=model.predict_classes(X_test)

In [115]:
from sklearn.metrics import confusion_matrix

In [116]:
confusion_matrix(y_test,y_pred)

array([[751, 320],
       [243, 777]])

In [117]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7307508369201339